In [1]:
import ee

import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import date
from datetime import timedelta
import zipfile
import os
import requests

import numpy as np

import geopy.distance
!pip install geopandas
import geopandas  as gpd
from shapely.geometry import Point, LineString, Polygon


# Import specific function 'from_epsg' from fiona module
from fiona.crs import from_epsg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 16.7 MB 10.9 MB/s 
     |████████████████████████████████| 6.3 MB 36.5 MB/s 


In [12]:
# connect to google drive
from google.colab import drive  

# drive roots
DRIVE_MOUNT = '/content/drive'
drive.mount(DRIVE_MOUNT, force_remount=True)

# set up project directories
PROJECT_DIR = DRIVE_MOUNT + "/My Drive/Volcano_datasets/"
TMP_DIR = PROJECT_DIR + "TempDIR/"
targets = pd.read_excel(PROJECT_DIR + "Sample Eruptions.xlsx")

OUTPUT_SHAPE_DIR = PROJECT_DIR+"output_damage_shapes/"

Mounted at /content/drive


In [25]:
distances = [1,5]
newdata = gpd.GeoDataFrame()
id = 0
for ivol in range(targets.shape[0]):
  volc = targets.iloc[ivol]
  vloc = Point( volc["Longitude"],volc["Latitude"])
  v_name = re.sub(r"[,.]","",re.sub(r"\s+", "_",volc["Volcano Name"]))
  erupt_start_date = date(volc["Start Year"].astype(int),
                            volc["Start Month"].astype(int),
                            volc["Start Day"].astype(int))
  
  for dist in distances:
    pts = []
    for bearing in range(0,350,10):
      pt =  geopy.distance.distance(kilometers=dist * np.random.rand())\
                   .destination((volc["Latitude"], volc["Longitude"]), bearing=bearing)[:2]
      pts.append([pt[1],pt[0]])
    poly = Polygon(pts)
    newdata.loc[id,'geometry'] = poly
    newdata.loc[id,"Volc_Name"] = v_name
    newdata.loc[id,"Volc_Num"] = volc["Volcano Number"]
    newdata.loc[id,"Erupt_Num"] = volc["Eruption Number"]
    newdata.loc[id,"Est_Date"] = erupt_start_date.isoformat()
    newdata.loc[id,"layer"] = dist
    id = id +1
    
print(newdata)
newdata.set_crs('EPSG:3857')
output_file = f'{OUTPUT_SHAPE_DIR}/all_data.shp'
newdata.to_file(output_file)

                                             geometry              Volc_Name  \
0   POLYGON ((130.21700 30.44502, 130.21823 30.449...       Kuchinoerabujima   
1   POLYGON ((130.21700 30.45285, 130.21939 30.454...       Kuchinoerabujima   
2   POLYGON ((120.99300 14.01059, 120.99358 14.005...                   Taal   
3   POLYGON ((120.99300 14.04334, 120.99486 14.012...                   Taal   
4   POLYGON ((-91.55000 -0.36779, -91.54864 -0.362...             Fernandina   
5   POLYGON ((-91.55000 -0.34811, -91.54233 -0.326...             Fernandina   
6   POLYGON ((-85.32400 10.83611, -85.32329 10.833...     Rincon_de_la_Vieja   
7   POLYGON ((-85.32400 10.84845, -85.32112 10.846...     Rincon_de_la_Vieja   
8   POLYGON ((130.30500 30.80108, 130.30603 30.798...                  Kikai   
9   POLYGON ((130.30500 30.82634, 130.31122 30.823...                  Kikai   
10  POLYGON ((123.50800 -8.26762, 123.50855 -8.270...              Lewotolok   
11  POLYGON ((123.50800 -8.27371, 123.51